# **Импорты**

In [ ]:
%pip install wandb ruff datasets jsonlines tensorflow tensorboard accelerate transformers

In [ ]:
# from google.colab import userdata
# userdata.get('HF_TOKEN')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from transformers import get_linear_schedule_with_warmup

from tqdm.auto import tqdm

import wandb

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

from datasets import load_dataset, Dataset

from torch.amp import autocast

from accelerate import Accelerator

# **Датасет - чек**

In [ ]:
# checkpoint = "aeonium/Aeonium-v1-BaseWeb-1B"
checkpoint = "AlexWortega/wortegaLM-1b"
tok = AutoTokenizer.from_pretrained(checkpoint, padding_side='right')
mod = AutoModelForCausalLM.from_pretrained(checkpoint)

In [ ]:
# train_data = data[0]

# emb = []
# for i in tqdm(range(len(train_data))):
#   instructions = train_data[i]["instrucrions"]
#   inputs = train_data[i]["inputs"]
#   targets = train_data[i]["targets"]

#   train_row = f"[BOR] {inputs}, {targets} [EOR] {tok.eos_token}"
#   embeded_row = tok.encode(train_row, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')

#   emb += [embeded_row]
# emb[:5]

In [ ]:
# alpaca_ds = load_dataset("IlyaGusev/ru_turbo_alpaca")
# train_data = alpaca_ds["train"]

# emb = []
# for i in tqdm(range(len(train_data))):
#     instruction = train_data[i]["instruction"]
#     input = train_data[i]["input"]
#     output = train_data[i]["output"]
    
#     train_row = f"{tok.bos_token}[INST]{instruction} {input}[/INST]{output}{tok.eos_token}"
    
#     if len(train_row)<=1024:
#         embeded_row = tok.encode(train_row, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
#         emb += [embeded_row]

In [ ]:
train_data = load_dataset("d0rj/OpenOrca-ru", split="train")

In [ ]:
removed_duplicates_dataset = Dataset.from_pandas(pd.DataFrame(train_data).drop_duplicates())

In [ ]:
train_data

In [ ]:
removed_duplicates_dataset

In [ ]:
c = 0
emb =[]
for i in tqdm(range(len(removed_duplicates_dataset))):
    instruction = removed_duplicates_dataset[i]["system_prompt"]
    input = removed_duplicates_dataset[i]["question"]
    output = removed_duplicates_dataset[i]["response"]
    
    train_row = f"{tok.bos_token}[INST]{instruction} {input}[/INST]{output}{tok.eos_token}"
    
    if len(train_row.split())<=700:
        embeded_row = tok.encode(train_row, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
        emb += [embeded_row]
        c+=1
    if c == 400_000:
        break

In [ ]:
mod.resize_token_embeddings(len(tok))

gen_kwargs = {
        "min_length": 20,
        "max_new_tokens": 100,
        "top_k": 50,
        "top_p": 0.7,
        "do_sample": True,  
        "early_stopping": True,
        "no_repeat_ngram_size": 2,
        "eos_token_id": tok.eos_token_id,
        "pad_token_id": tok.eos_token_id,
        "use_cache": False,
        "repetition_penalty": 1.5,  
        "length_penalty": 1.2,  
        "num_beams": 4,
        "num_return_sequences": 1
    }

text = "Какого цвета трава?"

t = tok.encode(text, add_special_tokens=False, return_tensors='pt')
print(t)
g = mod.generate(t, **gen_kwargs)
generated_sequences = tok.batch_decode(g, skip_special_tokens=False)

# outputs = mod.generate(**input_ids, **gen_kwargs)
print(generated_sequences)


# tok.decode(input_ids["input_ids"][0])

In [ ]:
tok.pad_token = tok.eos_token
tok.add_special_tokens({"additional_special_tokens": ["[INST]", "[/INST]"]})
mod.resize_token_embeddings(len(tok))

In [ ]:
emb_dataset= torch.utils.data.ConcatDataset([emb])
train_dataset = DataLoader(emb_dataset, batch_size = 16, shuffle = True)

# **LLM**

In [ ]:
class EMA(nn.Module):
  def __init__(self, decay: float):
        super().__init__()
        self.decay = decay
        self.shadow_params = {}
  def forward(self, model: nn.Module):
    for name, params in model.named_parameters():
      if params.requires_grad:
        if name not in self.shadow_params:
          self.shadow_params[name] = params.data.clone()
        else:
          # shadow_variable -= (1 - decay) * (shadow_variable - variable)
          self.shadow_params[name] -= (1 - self.decay) * (self.shadow_params[name] - params)
        params.data = self.shadow_params[name]

ema = EMA(0.5)

In [ ]:
def freeze(model: nn.Module):
  for param in model.parameters():
    param.requires_grad = False

In [ ]:
# accelerator = Accelerator()

training_steps = len(train_dataset)
optimizer = AdamW(mod.parameters())
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=training_steps//10, num_training_steps=training_steps)

wandb.login(key="1744a8a36a11513989635282773e3dc518b05f1c", relogin=True)
wandb.init(sync_tensorboard=True, name='train', project="hse-project", entity="aid_")

device = "cuda:0" if torch.cuda.is_available() else "cpu"
mod.to(device)

# freeze(mod)

# mod, optimizer, training_dataloader, scheduler = accelerator.prepare(
#     mod, optimizer, train_dataset, scheduler
# )

mod.train()
def one_epoch(model, data):
  for batch in tqdm(data):
    batch = batch.view(batch.shape[0], batch.shape[-1])

    t = batch.to(device)

    optimizer.zero_grad()

    with autocast(device_type="cuda"):
      loss = model(input_ids=t, labels=t)['loss']
      wandb.log({"loss": loss})

    # accelerator.backward(loss)
    loss.backward()
    optimizer.step()
    scheduler.step()

    del t
    torch.cuda.empty_cache()
    # ema(model)

  model.eval()

In [ ]:
for epoch in tqdm(range(5)):
  one_epoch(mod, train_dataset)
  # mod.save_pretrained(f'/home/jovyan/vqa_project/llm_tune/Aeonium-v1-BaseWeb-1B-orca-{epoch+1}epoch')
  mod.save_pretrained(f'/home/jovyan/vqa_project/llm_tune/WortegaLM-{epoch+1}epoch')

In [ ]:
input_text = f"[INST]Какого цвета небо?[/INST]"

gen_kwargs = {
        "min_length": 20,
        "max_new_tokens": 100,
        "top_k": 50,
        "top_p": 0.7,
        "do_sample": True,  
        "early_stopping": True,
        "no_repeat_ngram_size": 2,
        "eos_token_id": tok.eos_token_id,
        "pad_token_id": tok.eos_token_id,
        "use_cache": False,
        "repetition_penalty": 1.5,  
        "length_penalty": 1.2,  
        "num_beams": 4,
        "num_return_sequences": 1
    }

t = tok.encode(input_text, add_special_tokens=False, return_tensors='pt').to("cuda:1")
print(t)
g = mod.generate(t, **gen_kwargs)
generated_sequences = tok.batch_decode(g, skip_special_tokens=False)
print(generated_sequences)

In [ ]:
help(mod.generate)

# **Experiments with diff LLM**

**Gemma-2b**

In [ ]:
# checkpoint = "google/gemma-2b"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForCausalLM.from_pretrained(checkpoint)

# input_text = "Какого ."
# input_ids = tokenizer(input_text, return_tensors="pt")

# outputs = model.generate(**input_ids)
# print(tokenizer.decode(outputs[0]))

# del tokenizer
# del model

**Phi-1.5**

In [ ]:
# checkpoint = "microsoft/phi-1_5"

# # config = AutoConfig.from_pretrained(checkpoint, max_new_tokens = 128)
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForCausalLM.from_pretrained(checkpoint)

# input_text = "How many legs does a horse have?"
# input_ids = tokenizer(input_text, return_tensors="pt")

# print(input_ids)

# outputs = model.generate(**input_ids, max_new_tokens = 32)

# print(outputs)

# print(tokenizer.decode(outputs[0]))

In [ ]:
# rus_inp_text = "Какого цвета небо?"
# rus_ids = tokenizer(rus_inp_text, return_tensors="pt")
# rus_out = model.generate(**rus_ids, max_length = 256)

In [ ]:
# print(tokenizer.decode(rus_out[0]))

Phi-3

In [ ]:
# checkpoint = "microsoft/Phi-3-mini-128k-instruct"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForCausalLM.from_pretrained(checkpoint)

# input_text = "Какого цвета небо?"
# input_ids = tokenizer(input_text, return_tensors="pt")

# outputs = model.generate(**input_ids)
# print(tokenizer.decode(outputs[0]))

# del tokenizer
# del model

# **Image encoders**

In [ ]:
from transformers import ViTImageProcessor, ViTModel

checkpoint = "google/vit-base-patch16-224-in21k"
processor = ViTImageProcessor.from_pretrained(checkpoint)
model = ViTModel.from_pretrained(checkpoint)

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

# img = Image.open("/content/pict.png").convert("RGB")
transform = transforms.Compose([
    transforms.Resize((254, 254)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

# data = transform(img)

In [ ]:
# inputs = processor(data, return_tensors="pt")

In [ ]:
# inputs["pixel_values"]

In [ ]:
# outputs = model(inputs["pixel_values"])

In [ ]:
# outputs

In [ ]:
training_steps = 1000
# optimizer = AdamW(mod.parameters())
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=training_steps//10, num_training_steps=training_steps)


def train_step(model, train_dl):
    train_loss = 0
    model.train()
    for bid, (batch, _) in enumerate(train_dl):
        if bid % 100 == 0:
            print("...{:d} training steps complete".format(bid))
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch, return_loss=True)
        loss = outputs.loss
        train_loss += loss.detach().cpu().numpy()
        loss.backward()

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    print("...{:d} training steps COMPLETE".format(bid))
    return train_loss


def eval_step(model, eval_dl):
    model.eval()
    val_loss, val_acc, num_examples = 0, 0, 0
    for bid, (batch, _) in enumerate(eval_dl):
        if bid % 100 == 0:
            print("... {:d} validation steps complete".format(bid))
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch, return_loss=True)

        loss = outputs.loss
        val_loss += loss.detach().cpu().numpy()

        logits_per_image = outputs.logits_per_image
        probs = logits_per_image.softmax(dim=1)
        predictions = torch.argmax(probs, dim=-1)
        labels = torch.arange(len(predictions)).to(device)

        accuracy = torch.sum(predictions == labels)
        num_examples += len(predictions)
        val_acc += accuracy

    print("... {:d} validation steps COMPLETE".format(bid))
    val_acc = val_acc.detach().cpu().numpy() / num_examples
    return val_loss, val_acc

# TEST

In [ ]:
# my_model = "'/home/jovyan/vqa_project/llm_tune/WortegaLM-{5}epoch'"
# tok = AutoTokenizer.from_pretrained(checkpoint) # от 1 эпохи стартуем
# mod = AutoModelForCausalLM.from_pretrained(my_model, local_files_only=True) # от 1 эпохи стартуем

# tok.pad_token = tok.eos_token
# tok.add_special_tokens({"additional_special_tokens": ["[INST]", "[/INST]"]})
# mod.resize_token_embeddings(len(tok))
# mod.to("cuda:1")

In [ ]:
input_text = f"{tok.bos_token}[INST]Какого цвета небо?[/INST]"

gen_kwargs = {
        "min_length": 20,
        "max_new_tokens": 100,
        "top_k": 50,
        "top_p": 0.7,
        "do_sample": True,  
        "early_stopping": True,
        "no_repeat_ngram_size": 2,
        "eos_token_id": tok.eos_token_id,
        "pad_token_id": tok.eos_token_id,
        "use_cache": True,
        "repetition_penalty": 1.5,  
        "length_penalty": 1.2,  
        "num_beams": 4,
        "num_return_sequences": 1
    }
    
t = tok.encode(input_text, add_special_tokens=False, return_tensors='pt').to(device)
g = mod.generate(t, **gen_kwargs)
generated_sequences = tok.batch_decode(g, skip_special_tokens=False)

print(generated_sequences)

In [ ]:
print(tok.decode(outputs[0], skip_special_tokens=False))

In [ ]:
outputs[0]

In [ ]:
tok.decode(tok(input_text, return_tensors="pt")["input_ids"][0], skip_special_tokens=True)